In [42]:
#M.G. 5/23/17, Most recent update 6/1/17, Currently running excessively slowly

t = var('t') #Global time variable
y(x) = sqrt(3)*x + 1 #Gives y-coordinate as a function of x-coordinate for e3 trajectory w/ initial conditions x0 = 0, y0 = 1
z(x, y) = x + y*I


def linfrac(x, y): #Mobius transformation R.z, where z = x + yi, and R as defined in paper
    a = N(cos(pi/3)); b = N(-sin(pi/3)); c = N(sin(pi/3)); d = N(cos(pi/3)); z = x + y*I
    return N((a*z + b)/(c*z + d))

def inverse_linfrac(x, y):
    a = N(cos(-pi/3)); b = N(-sin(-pi/3)); c = N(sin(-pi/3)); d = N(cos(-pi/3)); z = x + y*I
    return N((a*z + b)/(c*z + d))

def e3(x_init, y_init, t):
    m = N(1/sqrt(3)); c0 = N(x_init + m); c1 = N(y_init/c0)
    x = -m + c0*e^(c1*t)
    y = c0*c1*e^(c1*t)
    z = x + y*I
    return N(z)

def e2(x_init, y_init, t):
    m = N(-1/sqrt(3)); c0 = N(x_init + m); c1 = N(y_init/c0)
    x = -m + c0*e^(c1*t)
    y = c0*c1*e^(c1*t)
    z = x + y*I
    return N(z)

def e1(x_init, y_init, t):
    lz_init = linfrac(x_init, y_init)
    lz_t = e3(lz_init.real(), lz_init.imag(), t)
    return N(inverse_linfrac(lz3.real(), lz3.imag()))
    
def get_time(x_init, y_init, x_fin, y_fin, t): #y_fin only used to determine the sign of m
    if y_init <= y_fin:
        m = 1/sqrt(3) 
    else:
        m = -1/sqrt(3)
    s = solve(x_fin == -m + (x_init + m)*e^((y_init/(x_init + m))*t), t)
    return s
    
def control_trajectory(x1): #The input here should be the terminal x-point you choose after initially traveling along e3; everything is determined from here
    x0 = 0; y0 = 1
    z0 = z(x0, y0)
    y1 = y(x1)
    z1 = z(x1, y1)
    t1 = get_time(x0, y0, x1, y1, t)[0].right() #Only one solution in list (thus [0]) and we want the r.h.s. (thus .right())
    x2 = -x1; y2 = y1 #By symmetry
    z2 = z(x2, y2)
    Rz1 = linfrac(x1, y1); Rz2 = linfrac(x2, y2)
    Rx1 = Rz1.real(); Ry1 = Rz1.imag()
    Rx2 = Rz2.real(); Ry2 = Rz2.imag()
    t2 = get_time(Rx1, Ry1, Rx2, Ry2, t)[0].right()
    x3 = x0; y3 = y0 #Notational convenience; there is no fundamental difference between (x3, y3) and (x0, y0)
    z3 = z(x3, y3)
    t3 = get_time(x2, y2, x3, y3, t)[0].right()
    solz = [z1, z2, z3]
    solt = [t1, t2, t3]
    sol = [N(solz[0]), N(solt[0]), N(solz[1]), N(solt[1]), N(solz[2]), N(solt[2])] #It takes time ti to get from z(i-1) to zi, i = 1, 2, 3
    return sol #read sol as three sets of ordered pairs: (solz[j], solt[j]), j = 0, 1, 2; we do not return it formatted as such in order to retain ease of access to elements

def t1(x1):
    return control_trajectory(x1)[1]

def t2(x1):
    return control_trajectory(x1)[3]

def t3(x1):
    return control_trajectory(x1)[5]

#def x(x_init, y_init, x_fin, y_fin, t):
#    t1 = 
#    if 0 <= t <= get_time(x_init, y_init, x_fin, y_fin, t)[0].right():
#        return N(-m + (x_init + m)*e^((y_init/(x_init + m))*t))
#    elif get_time(x_init, y_init, x_fin, y_fin, t)[0].right() < t < 

def path(x1, t):
    t1 = control_trajectory(x1)[1]
    t2 = control_trajectory(x1)[3]
    t3 = control_trajectory(x1)[5]
    z1 = control_trajectory(x1)[0]
    z2 = control_trajectory(x1)[2]
    z3 = control_trajectory(x1)[4]
    Rz1 = linfrac(z1.real(), z1.imag())
    if 0 <= t <= t1:
        return [-1/sqrt(3) + (1/sqrt(3))*e^(sqrt(3)*t), e^(sqrt(3)*t)]
    elif t1 < t < t1 + t2:
        return [-1/sqrt(3) + (Rz1.real() + 1/sqrt(3))*e^((Rz1.imag()/(Rz1.real() + 1/sqrt(3)))*t), Rz1.imag()*e^((Rz1.imag()/(Rz1.real() + 1/sqrt(3)))*t)]
    elif t1 + t2 <= t <= t1 + t2 + t3:
        return [1/sqrt(3) + (z2.real() - 1/sqrt(3))*e^((z2.imag()/(z2.real() - 1/sqrt(3)))*t), z2.imag()*e((z2.imag()/(z2.real - 1/sqrt(3)))*t)]
    else:
        print("Error")
    
#def cost(x1):
    

#a = 1/sqrt(3) - .1
#for i in range(0, 1000):
#    print(z1(0, 1, i/1000))

#parametric_plot((lambda t: z1(a, 3, t).real(), lambda t: z1(a, 3, t).imag()), (t, 0, .2))
#print(control_trajectory(0.1))
#parametric_plot((lambda t: z3(a, 3, t).real(), lambda t: z3(a, 3, t).imag()), (t, 0, .2))
#control_trajectory(0.1)
#for i in range(0, 1000):
#    print(N(get_time(0, 1, i, sqrt(3)*i + 1, t)[0].right()))

TypeError: ECL says: Console interrupt.